<a href="https://colab.research.google.com/github/wesslen/hdc-agenda-retrieval/blob/main/notebooks/gemini_structured_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!uv pip install --system "google-genai>=1"

Using Python 3.11.11 environment at /usr
Resolved 16 packages in 736ms
⠋ Preparing packages... (0/1)
⠋ Preparing packages... (0/1)
⠋ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
⠙ Preparing packages... (0/1)
Prepared 1 package in 32ms
Uninstalled 1 package in 20ms
Installed 1 package in 3ms
 - google-genai==0.8.0
 + google-genai==1.0.0


In [2]:
from google import genai
from google.colab import userdata
# Create a client
api_key = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=api_key)

# Define the model you are going to use
model_id =  "gemini-2.0-flash"

In [4]:
!wget -O agenda.pdf https://github.com/wesslen/hdc-agenda-retrieval/raw/main/data/agenda/2025/00_feburary2025_agenda_final-c.pdf

--2025-02-10 01:49:52--  https://github.com/wesslen/hdc-agenda-retrieval/raw/main/data/agenda/2025/00_feburary2025_agenda_final-c.pdf
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wesslen/hdc-agenda-retrieval/main/data/agenda/2025/00_feburary2025_agenda_final-c.pdf [following]
--2025-02-10 01:49:52--  https://raw.githubusercontent.com/wesslen/hdc-agenda-retrieval/main/data/agenda/2025/00_feburary2025_agenda_final-c.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167690 (164K) [application/octet-stream]
Saving to: ‘agenda.pdf’

agenda.pdf          100%[===================>] 163.76K  --.-KB/

In [5]:
agenda_pdf = client.files.upload(file="agenda.pdf", config={'display_name': 'agenda'})

In [6]:
file_size = client.models.count_tokens(model=model_id,contents=agenda_pdf)
print(f'File: {agenda_pdf.display_name} equals to {file_size.total_tokens} tokens')
# File: agenda equals to 1350 tokens

File: agenda equals to 1350 tokens


In [7]:
from pydantic import BaseModel, Field

def extract_structured_data(file_path: str, model: BaseModel):
    # Upload the file to the File API
    file = client.files.upload(file=file_path, config={'display_name': file_path.split('/')[-1].split('.')[0]})
    # Generate a structured response using the Gemini API
    prompt = f"Extract the structured data from the following PDF file"
    response = client.models.generate_content(model=model_id, contents=[prompt, file], config={'response_mime_type': 'application/json', 'response_schema': model})
    # Convert the response to the pydantic model and return it
    return response.parsed

In [13]:
from pydantic import BaseModel, Field
from typing import Optional, List
from enum import Enum

class ApplicationType(str, Enum):
    CONSENT = "CONSENT"
    ROOF_REPLACEMENT = "ROOF REPLACEMENT"
    FENESTRATION_CHANGES = "FENESTRATION CHANGES"
    ADDITIONS = "ADDITIONS"
    DEMOLITION = "DEMOLITION"
    WINDOW_REPLACEMENT = "WINDOW REPLACEMENT"
    ADU = "ACCESSORY DWELLING UNIT"
    FRONT_PORCH_CHANGES = "FRONT PORCH CHANGES"

class ApplicationStatus(str, Enum):
    NEW = "NEW"
    NOT_HEARD = "NOT HEARD"
    CONTINUED = "CONTINUED"
    CONSENT = "CONSENT"

class Applicant(BaseModel):
    first_name: str
    last_name: str

class Property(BaseModel):
    address: str
    pid: str
    district: str

class HDCApplication(BaseModel):
    application_id: str = Field(..., description="HDC Application ID (e.g., HDCRMI-2025-00007)")
    applicants: List[Applicant]
    property: Property
    application_type: ApplicationType
    status: ApplicationStatus
    status_details: Optional[str] = Field(None, description="Additional status information like meeting dates")

    class Config:
        use_enum_values = True

result = extract_structured_data("agenda.pdf", HDCApplication)
print(type(result))


<class '__main__.HDCApplication'>


In [15]:
result.model_dump()

{'application_id': 'HDCRMI-2025-00007',
 'applicants': [{'first_name': 'Debra', 'last_name': 'Glennon'}],
 'property': {'address': '1936 Park Rd',
  'pid': '12108705',
  'district': 'Dilworth'},
 'application_type': 'CONSENT',
 'status': 'CONSENT',
 'status_details': None}